In [588]:
import numpy as np #numpy for mathematical purpose
randchoice = np.random.choice

In [589]:
class Graph:
    
    num_of_node = 4 #number of node
    
    #city graph to save vertices and edges of the problem
    city_graph = [[ 0, 5, 15, 4],
                  [ 5, 0,  4, 8],
                  [15, 4,  0, 1],
                  [ 4, 8,  1, 0]]

    #pheromone graph to assign pheromone level on designated edges
    pheromone = [[0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0]]
    
    @classmethod
    def reset(cls):
        cls.pheromone = [[0.0, 0.0, 0.0, 0.0],
                         [0.0, 0.0, 0.0, 0.0],
                         [0.0, 0.0, 0.0, 0.0],
                         [0.0, 0.0, 0.0, 0.0]]

In [602]:
class Ant:
    
    def __init__(self, start_node):
        
        self.start_node = start_node
        self.unvisited_node = []     #list of unvisited node
        self.nodes_probability = []  #probability of nodes to be visited
        self.nodes_cumulative = 0    #cumulative probability of all nodes
        self.generate_unvisited_node(start_node, Graph().num_of_node) # initialize unvisited node based on starting node
        self.route = []              #route that ant traversed
        self.distance_traveled = 0   #total distance ant traveled
        
        #temporary variable for pheromone on edge ants walk on
        self.pheromone =  [[0, 0, 0, 0],
                           [0, 0, 0, 0],
                           [0, 0, 0, 0],
                           [0, 0, 0, 0]]
        
        
    def generate_unvisited_node(self, start_node, num_of_node):
        for i in range (0, num_of_node):
            if(i != start_node):
                self.unvisited_node.append(i)
    
    
    def ant_travel(self, alpha, beta):
        prior_node = self.start_node
        visit_to = self.start_node
        
        print("Route: ")
        for i in range (0,Graph().num_of_node):
            current_node = prior_node
            print(str(current_node+1),end=" --> ") #print current node visited
            if self.unvisited_node:
                
                self.nodes_probability.clear()
                for i in range (0,len(self.unvisited_node)): #calculate cumulative probability
                    self.nodes_cumulative += self.calculate_probability(current_node,self.unvisited_node[i],self.unvisited_node,alpha,beta)
                
                if(self.nodes_cumulative != 0.0):
                    for i in range (0,len(self.unvisited_node)): #calculate each nodes probability
                        self.nodes_probability.append(self.calculate_probability(current_node,self.unvisited_node[i],self.unvisited_node,alpha,beta) / self.nodes_cumulative)
                
                #fail-save if sum of cumulative probability less or more than 1
                self.nodes_probability = np.array(self.nodes_probability)
                self.nodes_probability /= self.nodes_probability.sum() #re-normalized
                
                if(self.nodes_cumulative == 0):
                    visit_to = randchoice(self.unvisited_node)
                else:
                    visit_to = randchoice(self.unvisited_node, p=self.nodes_probability) #choose node done with numpy.random
                
                self.nodes_probability = self.nodes_probability.tolist()
                
                self.unvisited_node.remove(visit_to) #remove visited node from unvisited-node list
                prior_node = visit_to
                self.distance_traveled = self.distance_traveled + Graph().city_graph[current_node][visit_to] #sum of distance
                self.route.append([current_node,visit_to]) #record route
            else:
                self.distance_traveled = self.distance_traveled + Graph().city_graph[current_node][self.start_node] #sum of distance
                self.route.append([current_node,start_node]) #record route
                print(str(start_node+1))

    
    def calculate_probability(self, i, j, all_nodes, alpha, beta):
        probability = pow(Graph().pheromone[i][j],alpha) * pow((1/Graph().city_graph[i][j]),beta)
        return probability
    
    def lay_pheromone(self, Q):
        for i in range (0, len(self.route)):
            #update pheromone value (upper triangle)
            self.pheromone[self.route[i][0]][self.route[i][1]] = Q / self.distance_traveled
            #update pheromone value (lower triangle)
            self.pheromone[self.route[i][1]][self.route[i][0]] = Q / self.distance_traveled
        
        #uncomment 2 lines below to show ant's temporary pheromone value 
        #for i in range (0,Graph().num_of_node):
         #   print(self.pheromone[i])

In [603]:
class ACO:
    
    def __init__(self, number_of_ants, start_node, Q, Rho):
        
        self.ant = []
        self.Q = Q          #pheromone trail's constant
        self.Rho = Rho      #pheromone evaporation rate
        self.number_of_ants = number_of_ants #number of ants in population
        for i in range (0,number_of_ants):
            self.ant.append(Ant(start_node)) #instantiate ant objects
        
        #sum of all ants' pheromone on designated edges
        self.all_ants_pheromone = [[0, 0, 0, 0],
                                  [0, 0, 0, 0],
                                  [0, 0, 0, 0],
                                  [0, 0, 0, 0]]
    
    #control all ants' movement and tour
    def ants_travel(self, alpha, beta): 
        for i in range (0,self.number_of_ants):
            print("\nAnt " + str(i+1) + " travels: ")
            self.ant[i].ant_travel(alpha,beta)
            print("Total distance: " + str(self.ant[i].distance_traveled))
            self.ant[i].lay_pheromone(self.Q)
    
    #control all ant's pheromone trail
    def ants_pheromone(self):
        for i in range (0,self.number_of_ants):
            for j in range (0, Graph().num_of_node):
                for k in range (0, Graph().num_of_node):
                        self.all_ants_pheromone[j][k] += self.ant[i].pheromone[j][k]
                        
    #assign pheromone total value to problem domain
    def update_pheromone(self):
        self.ants_pheromone()
        #print("\nall:" + str(self.all_ants_pheromone))
        
        for j in range (0, Graph().num_of_node):
            for k in range (0, Graph().num_of_node):
                Graph().pheromone[j][k] = (1-self.Rho)*Graph().pheromone[j][k] + self.all_ants_pheromone[j][k]
                        

In [610]:
number_of_ants = 10    #number of ants
start_node = 0        #starting node index
Q = 1                 #pheromone's trail constant
Rho = 0.8             #pheromone evaporation rate
alpha = 0.5           #pheromone weighting parameter
beta = 0.5            #visibility weighting parameter

Graph().reset()

for i in range (0,20):
    print("\n--------------- ITERATION #" + str(i+1) + " -------------------------")
    aco = ACO(number_of_ants, start_node, Q, Rho)
    aco.ants_travel(alpha, beta)
    aco.update_pheromone()
    
    #uncomment 3 lines below to show total pheromone after i-th iteration
    #print("Total pheromone: \n")
    #for j in range (0,4):
     #   print(Graph().pheromone[j])


--------------- ITERATION #1 -------------------------

Ant 1 travels: 
Route: 
1 --> 3 --> 4 --> 2 --> 1
Total distance: 29

Ant 2 travels: 
Route: 
1 --> 4 --> 2 --> 3 --> 1
Total distance: 31

Ant 3 travels: 
Route: 
1 --> 4 --> 3 --> 2 --> 1
Total distance: 14

Ant 4 travels: 
Route: 
1 --> 2 --> 4 --> 3 --> 1
Total distance: 29

Ant 5 travels: 
Route: 
1 --> 2 --> 3 --> 4 --> 1
Total distance: 14

Ant 6 travels: 
Route: 
1 --> 3 --> 2 --> 4 --> 1
Total distance: 31

Ant 7 travels: 
Route: 
1 --> 3 --> 4 --> 2 --> 1
Total distance: 29

Ant 8 travels: 
Route: 
1 --> 3 --> 4 --> 2 --> 1
Total distance: 29

Ant 9 travels: 
Route: 
1 --> 3 --> 2 --> 4 --> 1
Total distance: 31

Ant 10 travels: 
Route: 
1 --> 4 --> 2 --> 3 --> 1
Total distance: 31

--------------- ITERATION #2 -------------------------

Ant 1 travels: 
Route: 
1 --> 4 --> 2 --> 3 --> 1
Total distance: 31

Ant 2 travels: 
Route: 
1 --> 4 --> 3 --> 2 --> 1
Total distance: 14

Ant 3 travels: 
Route: 
1 --> 2 --> 4 --> 3 --